<a href="https://colab.research.google.com/github/Kate-Cher/BI-2020-2021_Python/blob/SQL_hw/genotype_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genotyping data database

In [42]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Creating db with 2 tables

In [4]:
import csv
import pandas as pd
import sqlite3

In [5]:
# Create DB
connection = sqlite3.connect('genotype.db')

In [6]:
#Create table
query = '''CREATE TABLE meta 
            ("dna_chip_id" TEXT PRIMARY KEY, 
            "breed" TEXT, 
            "sex" TEXT)'''
connection.execute(query)
connection.commit()

In [ ]:
# Read data
metadata = pd.read_csv('/content/metadata.csv', index_col=0)
genstudio = pd.read_csv('/content/genstudio.csv', index_col=0)
genstudio = genstudio.rename(columns={ 'Sample ID' : 'dna_chip_id'})

In [8]:
# Insert DF to DB
metadata.to_sql('meta', con=connection, if_exists='append', index=False, chunksize=10000)

In [9]:
# SNP's analyze
genstudio[['dna_chip_id']].value_counts()

dna_chip_id       
202341831114R08C01    68516
202341831127R03C01    68516
202341831114R01C02    68516
202341831114R02C01    68516
202341831114R02C02    68516
202341831114R03C01    68516
202341831114R03C02    68516
202341831114R04C01    68516
202341831114R04C02    68516
202341831114R05C01    68516
202341831114R05C02    68516
202341831114R06C01    68516
202341831114R06C02    68516
202341831114R07C01    68516
202341831114R07C02    68516
202341831114R01C01    68516
202341831114R08C02    68516
202341831114R09C01    68516
202341831114R09C02    68516
202341831114R10C01    68516
202341831114R10C02    68516
202341831114R11C01    68516
202341831114R11C02    68516
202341831114R12C01    68516
202341831114R12C02    68516
202341831127R01C01    68516
202341831127R01C02    68516
202341831127R02C01    68516
202341831127R02C02    68516
202341831127R03C02    13036
dtype: int64

As we can see, there are some problems in 202341831127R03C02 DNA chip ID, so probably we should delete this ID form db

In [10]:
# Create db
query = '''CREATE TABLE genotype
            ("genotyping_id" INTEGER PRIMARY KEY, 
            "SNP Name" TEXT, 
            "SNP Index" INTEGER, 
            "SNP Aux" INTEGER, 
            "dna_chip_id" TEXT, 
            "SNP" TEXT, 
            "Allele1 - Top" TEXT, 
            "Allele2 - Top" TEXT, 
            "Allele1 - Forward" TEXT, 
            "Allele2 - Forward" TEXT, 
            "Allele1 - AB" TEXT, 
            "Allele2 - AB" TEXT, 
            "Chr" TEXT, 
            "Position" TEXT, 
            "GC Score" REAL, 
            "GT Score" REAL, 
            "Theta" REAL, 
            "R" REAL, 
            "B Allele Freq" REAL, 
            "Log R Ratio" REAL,
            CONSTRAINT fk_metadata
                FOREIGN KEY ("dna_chip_id")
                REFERENCES meta("dna_chip_id")
                ON DELETE CASCADE)'''
connection.execute(query)
connection.commit()


In [11]:
# Insert genotype data to db
genstudio.to_sql('genotype', con=connection, if_exists='append', index=False, chunksize=10000)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


## Updating DB

Probably its more useful to have all fields the same language, so we've update DB 

In [26]:
query = """
UPDATE meta 
SET breed = 'D' 
WHERE breed = 'Д'
"""
connection.execute(query)
connection.commit()

## Editing db timeout

In [12]:
# Timeout change for editing db
def read_sqlite_table():
    try:
        sqlite_connection= sqlite3.connect('genotype.db', timeout=20)
        cursor = sqlite_connection.cursor()
        print("Подключен к SQLite")

        sqlite_select_query = """SELECT count(*) from genotype"""
        cursor.execute(sqlite_select_query)
        total_rows = cursor.fetchone()
        print("Всего строк:  ", total_rows)
        cursor.close()

    except sqlite3.Error as error:
        print("Ошибка при подключении к sqlite", error)

read_sqlite_table()

Подключен к SQLite
Всего строк:   (2000000,)


## Deleting bad DNA chip ID
We've noticed some problems with one of SNP's. So we want to delete it from both metadata and genotype

In [18]:
query = '''
    PRAGMA foreign_keys=1'''
connection.execute(query)

query = '''
    DELETE FROM meta
    WHERE "dna_chip_id" = "202341831114R02C01"'''
connection.execute(query)
connection.commit()

## Fetching data

As we deleted bad chip ID let's check that it was right and try to find this bad chip ID in our db 

In [24]:
query = '''
    SELECT * FROM meta
    WHERE "dna_chip_id" = "202341831114R02C01"'''
find = connection.execute(query)
connection.commit()  
print(find.fetchall())

[]


As we expected there is no such ID in DB now

Now we want to check correctness of our joining data, so let's fetch 3 records from db.

In [25]:
query = '''
    SELECT * FROM meta
    JOIN genotype ON genotype."dna_chip_id" = meta."dna_chip_id"
    LIMIT 5'''
res = connection.execute(query)
for line in res.fetchall():
    print(line)

('202341831114R01C01', 'Д', 'Хр', 1, '1_10573221', 1, 0, '202341831114R01C01', '[T/C]', '-', '-', '-', '-', '-', '-', '1', '10573221', 0.0, 0.0, 0.9420000000000001, 0.413, 1.0, 0.404)
('202341831114R01C01', 'Д', 'Хр', 2, '1_10673082', 2, 0, '202341831114R01C01', '[T/C]', 'A', 'A', 'T', 'T', 'A', 'A', '1', '10673082', 0.8272, 0.8076, 0.039, 0.968, 0.0, 0.3017)
('202341831114R01C01', 'Д', 'Хр', 3, '1_10723065', 3, 0, '202341831114R01C01', '[A/G]', 'A', 'A', 'T', 'T', 'A', 'A', '1', '10723065', 0.8316, 0.8107, 0.011000000000000001, 1.577, 0.0, 0.0388)
('202341831114R01C01', 'Д', 'Хр', 4, '1_11337555', 4, 0, '202341831114R01C01', '[A/G]', 'A', 'A', 'T', 'T', 'A', 'A', '1', '11337555', 0.3781, 0.7925, 0.045, 1.104, 0.0, 0.2761)
('202341831114R01C01', 'Д', 'Хр', 5, '1_11407894', 5, 0, '202341831114R01C01', '[A/G]', 'G', 'G', 'G', 'G', 'B', 'B', '1', '11407894', 0.9038, 0.867, 0.983, 1.122, 0.9994, 0.0022)


Also let's look at 15 first SNP's names with [T/C] SNP

In [39]:
query = '''
    SELECT "SNP name" FROM genotype
    WHERE "SNP"="[T/C]"
    LIMIT 15'''
res = connection.execute(query)
for line in res:
  print(line)

('1_10573221',)
('1_10673082',)
('1_11426075',)
('1_11445382',)
('1_13996200',)
('1_14638936',)
('1_17537210',)
('1_2812415',)
('1_294072400',)
('1_304970973',)
('1_305018214',)
('1_308334169',)
('1_309509770',)
('1_310058452',)
('1_32706368',)


## Delete records

We are not really interested in SNPs in X or Y chr, so let's delete this records

In [40]:
query = '''
    DELETE FROM genotype
    WHERE "Chr" = "X"'''
connection.execute(query)
connection.commit()
query = '''
    DELETE FROM genotype
    WHERE "Chr" = "Y"'''
connection.execute(query)
connection.commit()

In [41]:
# close database
connection.close()